In [1]:
import torch
import torch.nn as nn
import math
import random
import pandas as pd
import numpy as np

# nlp library of Pytorch
from torchtext import data
#from torchtext.legacy import data

import warnings as wrn
wrn.filterwarnings('ignore')
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cuda.deterministic = True

***

In [2]:
df = pd.read_csv("csgo_imputed_shifted_encoded_scaled.csv")
df[0:10]

,file,round,winner_side,ct_eq_val,t_eq_val,length_seconds,t_headshot_hits,t_damage,t_num_dmg_instances,t_wp_type_Unkown_damage,...,was_missing,next_round_winner,map_de_overpass,map_de_cache,map_de_inferno,map_de_mirage,map_de_train,map_de_dust2,map_de_cbble,map_de_nuke
0,esea_match_13770997.dem,0.000000,0.5,0.091762,0.084391,0.029389,0.326071,0.283685,0.219512,0.000202,...,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,esea_match_13770997.dem,0.034483,0.0,0.145759,0.497992,0.052491,0.250000,0.314077,0.206897,0.000000,...,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,esea_match_13770997.dem,0.068966,0.0,0.178017,0.493976,0.018862,0.250000,0.326029,0.224138,0.000000,...,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,esea_match_13770997.dem,0.103448,0.0,0.590203,0.605087,0.038276,0.166667,0.364542,0.431034,0.000000,...,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,esea_match_13770997.dem,0.137931,1.0,0.124253,0.528782,0.011292,0.083333,0.176627,0.241379,0.000000,...,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,esea_match_13770997.dem,0.172414,0.0,0.703704,0.659973,0.053603,0.333333,0.353918,0.224138,0.000000,...,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,esea_match_13770997.dem,0.206897,0.0,0.071685,0.655957,0.029565,0.500000,0.326693,0.137931,0.000000,...,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,esea_match_13770997.dem,0.241379,1.0,0.111111,0.717537,0.029528,0.166667,0.262948,0.241379,0.000000,...,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,esea_match_13770997.dem,0.275862,1.0,0.763441,0.465863,0.029991,0.083333,0.245684,0.241379,0.000000,...,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,esea_match_13770997.dem,0.310345,1.0,0.782557,0.244980,0.037266,0.250000,0.283533,0.172414,0.000000,...,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
ycols = ["file", "next_round_winner", "was_missing"]
Xcols = list(set(df.columns) - set(ycols))

***

In [4]:
# get all matches
matches = df["file"].unique()
len(matches)

14918

In [5]:
# how long are the matches?
df["file"].value_counts().value_counts()

30    2272
29    1432
28    1297
27    1269
26    1255
25    1247
24    1203
23    1087
22     938
21     887
20     750
19     543
18     408
17     220
16      76
15      31
14       3
Name: file, dtype: int64

In [6]:
# get highest round for each match
highest_rounds = df["file"].value_counts().to_dict()
highest_rounds

{'esea_match_13821681.dem': 30,
 'esea_match_13826687.dem': 30,
 'esea_match_13782613.dem': 30,
 'esea_match_13826691.dem': 30,
 'esea_match_13823757.dem': 30,
 'esea_match_13785683.dem': 30,
 'esea_match_13782620.dem': 30,
 'esea_match_13823761.dem': 30,
 'esea_match_13791016.dem': 30,
 'esea_match_13782625.dem': 30,
 'esea_match_13782610.dem': 30,
 'esea_match_13823768.dem': 30,
 'esea_match_13782629.dem': 30,
 'esea_match_13823770.dem': 30,
 'esea_match_13782632.dem': 30,
 'esea_match_13785666.dem': 30,
 'esea_match_13791007.dem': 30,
 'esea_match_13785664.dem': 30,
 'esea_match_13826693.dem': 30,
 'esea_match_13789408.dem': 30,
 'esea_match_13826654.dem': 30,
 'esea_match_13785699.dem': 30,
 'esea_match_13791025.dem': 30,
 'esea_match_13785710.dem': 30,
 'esea_match_13785705.dem': 30,
 'esea_match_13789397.dem': 30,
 'esea_match_13820608.dem': 30,
 'esea_match_13826704.dem': 30,
 'esea_match_13823750.dem': 30,
 'esea_match_13785698.dem': 30,
 'esea_match_13782608.dem': 30,
 'esea_m

In [7]:
# get the matches that are between 15 and 30 rounds long
min_match_len = 15
max_match_len = 30

In [8]:
train_files = []
valid_files = []
test_files =  []

# 15/15/70 split

for match_length in range(min_match_len, max_match_len):
    files = np.array([key for key, value in highest_rounds.items() if value == match_length])
    
    test_valid_size = int(np.ceil(len(files)*.15))
    train_size = len(files) - 2*test_valid_size

    test = np.random.choice(a=files, size=test_valid_size, replace=False)
    the_rest = list(set(files) - set(test))
    
    valid = np.random.choice(a=the_rest, size=test_valid_size, replace=False)
    the_rest = list(set(the_rest) - set(valid))
    
    train = np.array(the_rest)
    
    test_files.extend(test)
    valid_files.extend(valid)
    train_files.extend(train)
    
    print(f"match len:{match_length}, test:{len(test)}, valid:{len(valid)}, train:{len(train)}, all:{len(files)}")
            

match len:15, test:5, valid:5, train:21, all:31
match len:16, test:12, valid:12, train:52, all:76
match len:17, test:33, valid:33, train:154, all:220
match len:18, test:62, valid:62, train:284, all:408
match len:19, test:82, valid:82, train:379, all:543
match len:20, test:113, valid:113, train:524, all:750
match len:21, test:134, valid:134, train:619, all:887
match len:22, test:141, valid:141, train:656, all:938
match len:23, test:164, valid:164, train:759, all:1087
match len:24, test:181, valid:181, train:841, all:1203
match len:25, test:188, valid:188, train:871, all:1247
match len:26, test:189, valid:189, train:877, all:1255
match len:27, test:191, valid:191, train:887, all:1269
match len:28, test:195, valid:195, train:907, all:1297
match len:29, test:215, valid:215, train:1002, all:1432


In [9]:
test_files[0:10]

['esea_match_13823981.dem',
 'esea_match_13823396.dem',
 'esea_match_13788175.dem',
 'esea_match_13823502.dem',
 'esea_match_13782770.dem',
 'esea_match_13785068.dem',
 'esea_match_13824315.dem',
 'esea_match_13828977.dem',
 'esea_match_13788042.dem',
 'esea_match_13828899.dem']

['esea_match_13823981.dem',
 'esea_match_13823396.dem',
 'esea_match_13788175.dem',
 'esea_match_13823502.dem',
 'esea_match_13782770.dem',
 'esea_match_13785068.dem',
 'esea_match_13824315.dem',
 'esea_match_13828977.dem',
 'esea_match_13788042.dem',
 'esea_match_13828899.dem']

***

In [10]:
df[ycols]

,file,next_round_winner,was_missing
0,esea_match_13770997.dem,0.0,1
1,esea_match_13770997.dem,0.0,0
2,esea_match_13770997.dem,0.0,0
3,esea_match_13770997.dem,1.0,0
4,esea_match_13770997.dem,0.0,0
...,...,...,...
375327,esea_match_13829173.dem,0.0,0
375328,esea_match_13829173.dem,0.0,0
375329,esea_match_13829173.dem,0.0,0
375330,esea_match_13829173.dem,0.0,0


***

In [11]:
dfnp = df.to_numpy()

In [12]:
cols_dict = {value:index for index, value in enumerate(df.columns)}

In [13]:
file_index_dict = {}
# get the indices of each file
for file in train_files + valid_files + test_files:
    file_index_dict[file] = np.where(dfnp[:, cols_dict["file"]] == file)[0]

In [14]:
def format_data(num_rounds, dfnp, files, file_index_dict, cols_dict, shuffle):
    to_concat_X = []
    to_concat_y = []
    
    if shuffle:
        random.Random(SEED).shuffle(files)

    for file in files:
        this_file_indices = file_index_dict[file]
        
        # make sure match is long enough
        if len(this_file_indices) >= num_rounds:
            # get rounds 1,...,num_rounds
            this_rounds = dfnp[this_file_indices[0:num_rounds], :]
            
            this_X = torch.as_tensor(this_rounds[:, [cols_dict[x] for x in Xcols]].astype("float"), dtype=torch.float)
            to_concat_X.append(this_X)
            
            this_y = this_rounds[-1, cols_dict["next_round_winner"]]
            to_concat_y.append(this_y)
    
    # make into shape (N, L, Hin)
    X_tensor = torch.stack(to_concat_X, dim=0)
    
    # make into shape (N)
    y_tensor = torch.tensor(to_concat_y, dtype=torch.float)
    
    return X_tensor, y_tensor

In [15]:
def batch_formatted_data(batch_size, X, y):
    X_batches = list(torch.split(X, batch_size, dim=0))
    y_batches = list(torch.split(y, batch_size, dim=0))
    return X_batches, y_batches

In [16]:
def data_to_batches(num_rounds, batch_size, dfnp, files, file_index_dict, cols_dict, shuffle):
    X, y = format_data(num_rounds, dfnp, files, file_index_dict, cols_dict, shuffle)
    X_batches, y_batches = batch_formatted_data(batch_size, X, y)
    
    # make sure the last batch doesn't have just one observation
    if X_batches[-1].shape[0] > 1:
        return X_batches, y_batches
    else:
        return X_batches[:-1], y_batches[:-1]
    

In [17]:
def get_all_batches(batch_size, shuffle, dfnp, files, file_index_dict, cols_dict):
    all_X_batches = []
    all_y_batches = []
    min_round = 1
    max_round = 30
    
    for num_rounds in range(min_round, max_round):
        X_batches, y_batches = data_to_batches(num_rounds, batch_size, dfnp, files, file_index_dict, cols_dict, shuffle)
        
        all_X_batches.extend(X_batches)
        all_y_batches.extend(y_batches)
        
    return [*zip(all_X_batches, all_y_batches)]

***

In [18]:
# build lstm architecture
class LSTMNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, dropout, out_features):
        super(LSTMNet,self).__init__()
        self.GRU = nn.GRU(input_size=input_size,
                         hidden_size=hidden_size,
                         num_layers=num_layers,
                         dropout=dropout,
                         batch_first=True)
        
        self.dense = nn.Linear(in_features=num_layers*hidden_size, 
                               out_features=out_features)
        
        self.prediction = nn.Sigmoid()
        
    def forward(self,x):
        # Hin = num_features
        # Hout = hidden_size
#         print(f"x.shape: {x.shape}") # (N, L, Hin)
        
        output, h_n = self.GRU(x, None)
#         print(f"h_n.shape: {h_n.shape}") # (2, N, Hout)
#         print(f"output shape: {output.shape}") # (N, L, Hout)
        
        x = torch.transpose(h_n, 0, 1)
        x = torch.flatten(x, 1)
#         print(f"x.shape: {x.shape}") # (N, 2*Hout)
        
        h = self.dense(x)
#         print(f"h.shape: {h.shape}") # (N, 1)
        
        yhat = self.prediction(h)
#         print(f"yhat.shape: {yhat.shape}") # (N, 1)
        
        return yhat

In [19]:
def clf_perf(predictions, target):
    # predictions need to be rounded and masked
    rounded_preds = torch.round(predictions)
    
    mask = torch.where(target != 0.5)[0]
    
    masked_preds = rounded_preds[mask]
    masked_target = target[mask]
    
    tp = torch.sum((masked_preds == 1.0) & (masked_target == 1.0)).item()
    tn = torch.sum((masked_preds == 0.0) & (masked_target == 0.0)).item()
    fp = torch.sum((masked_preds == 1.0) & (masked_target == 0.0)).item()
    fn = torch.sum((masked_preds == 0.0) & (masked_target == 1.0)).item()
    
    return np.array([tp, tn, fp, fn])

In [20]:
def conf_matrix(tp_tn_fp_fn):
    mat = pd.DataFrame(index=["Predicted Positive", "Predicted Negative"], 
                       columns=["Actual Positive", "Actual Negative"],
                      dtype="float")
    
    mat.iloc[0,0] = tp_tn_fp_fn[0] # tp
    mat.iloc[1,1] = tp_tn_fp_fn[1] # tn
    mat.iloc[0,1] = tp_tn_fp_fn[2] # fp
    mat.iloc[1,0] = tp_tn_fp_fn[3] # fn
    
    return mat

In [21]:
def acc_from_conf_mat(conf_mat):
    # tp + tn / all
    return (conf_mat.iloc[0,0] + conf_mat.iloc[1,1])/conf_mat.sum().sum()

In [22]:
def train(model, batches, optimizer, criterion, device):
    epoch_loss = 0.0
    tp_tn_fp_fn = np.array([0,0,0,0])
    
    model.train()
    
    for batch in batches:
        X = batch[0]
        y = batch[1]
        
        # cleaning the cache of optimizer
        optimizer.zero_grad()
        
        # forward propagation and squeezing
        X = X.to(device)
        predictions = model(X).squeeze()
        
        # mask missing targets
        criterion.weight = torch.tensor([0.0 if x == 0.5 else 1.0 for x in y.flatten()], device=device)
        
        # computing loss / backward propagation
#         print(f"X.shape: {X.shape}, y.shape: {y.shape}, predictions.shape: {predictions.shape}")
        
        
        y = y.to(device)
        loss = criterion(predictions, y)
        loss.backward()
        
        # classification performance
        tp_tn_fp_fn += clf_perf(predictions, y)
        
        # updating params
        optimizer.step()
        
        epoch_loss += loss.item()
    
    conf_mat = conf_matrix(tp_tn_fp_fn)
    avg_loss = epoch_loss / len(batches)
    acc = acc_from_conf_mat(conf_mat)
    
    return avg_loss, acc, conf_mat

In [23]:
def evaluate(model, batches, criterion, device):
    epoch_loss = 0.0
    tp_tn_fp_fn = np.array([0,0,0,0])
    
    # deactivate the dropouts
    model.eval()
    
    # Sets require_grad flat False
    with torch.no_grad():
        for batch in batches:
            X = batch[0]
            y = batch[1]

            # forward propagation and squeezing
            X = X.to(device)
            predictions = model(X).squeeze()

            # mask missing targets
            criterion.weight = torch.tensor([0.0 if x == 0.5 else 1.0 for x in y.flatten()], device=device)

            # computing loss / backward propagation
            y = y.to(device)
            loss = criterion(predictions, y)

            # classification performance
            tp_tn_fp_fn += clf_perf(predictions, y)

            epoch_loss += loss.item()

    conf_mat = conf_matrix(tp_tn_fp_fn)
    avg_loss = epoch_loss / len(batches)
    acc = acc_from_conf_mat(conf_mat)
    
    return avg_loss, acc, conf_mat

***

In [45]:
device = torch.device("cuda:0")

In [46]:
input_size = len(Xcols)
dropout = 0.0
out_features = 1
n_epochs = 15

In [47]:
batch_size = 256
shuffle = True
n_layers = 2
hidden_size = 16
lr = 0.001

In [48]:
train_batches = get_all_batches(batch_size, shuffle, dfnp, train_files, file_index_dict, cols_dict)
valid_batches = get_all_batches(batch_size, shuffle, dfnp, valid_files, file_index_dict, cols_dict)

***

In [49]:
model = LSTMNet(input_size, hidden_size, n_layers, dropout, out_features)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
criterion = nn.BCELoss(reduction="mean")
criterion = criterion.to(device)

In [50]:
perf_log = pd.DataFrame(index = np.arange(1,n_epochs+1), 
                        columns = ["Train Loss", "Valid Loss", "Train+Valid Loss", "Test Loss", "Train Acc", "Valid Acc", "Train+Valid Acc", "Test Acc"])

In [51]:
for epoch in range(1,n_epochs+1):
    
    train_loss, train_acc, train_conf_mat = train(model, train_batches, optimizer, criterion, device)
    
    valid_loss, valid_acc, valid_conf_mat = evaluate(model, valid_batches, criterion, device)
    
    perf_log.loc[epoch, ["Train Loss", "Train Acc"]] = [train_loss, train_acc]

    perf_log.loc[epoch, ["Valid Loss", "Valid Acc"]] = [valid_loss, valid_acc]

    
    # Showing statistics
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print()

	Train Loss: 0.560 | Train Acc: 61.61%
	 Val. Loss: 0.553 |  Val. Acc: 63.46%

	Train Loss: 0.548 | Train Acc: 63.48%
	 Val. Loss: 0.550 |  Val. Acc: 63.61%

	Train Loss: 0.545 | Train Acc: 63.56%
	 Val. Loss: 0.546 |  Val. Acc: 63.65%

	Train Loss: 0.541 | Train Acc: 63.83%
	 Val. Loss: 0.540 |  Val. Acc: 63.77%

	Train Loss: 0.537 | Train Acc: 64.14%
	 Val. Loss: 0.539 |  Val. Acc: 63.99%

	Train Loss: 0.536 | Train Acc: 64.28%
	 Val. Loss: 0.539 |  Val. Acc: 64.16%

	Train Loss: 0.535 | Train Acc: 64.43%
	 Val. Loss: 0.538 |  Val. Acc: 64.45%

	Train Loss: 0.534 | Train Acc: 64.53%
	 Val. Loss: 0.538 |  Val. Acc: 64.46%

	Train Loss: 0.533 | Train Acc: 64.60%
	 Val. Loss: 0.538 |  Val. Acc: 64.54%

	Train Loss: 0.532 | Train Acc: 64.68%
	 Val. Loss: 0.537 |  Val. Acc: 64.62%

	Train Loss: 0.531 | Train Acc: 64.73%
	 Val. Loss: 0.537 |  Val. Acc: 64.59%

	Train Loss: 0.531 | Train Acc: 64.82%
	 Val. Loss: 0.537 |  Val. Acc: 64.57%

	Train Loss: 0.530 | Train Acc: 64.86%
	 Val. Loss: 

In [52]:
perf_log

,Train Loss,Valid Loss,Train+Valid Loss,Test Loss,Train Acc,Valid Acc,Train+Valid Acc,Test Acc
1,0.560175,0.553076,NaN,NaN,0.616096,0.634593,NaN,NaN
2,0.548298,0.550298,NaN,NaN,0.634768,0.636138,NaN,NaN
3,0.545292,0.545981,NaN,NaN,0.635576,0.636543,NaN,NaN
4,0.540648,0.54049,NaN,NaN,0.638334,0.637733,NaN,NaN
5,0.537382,0.53929,NaN,NaN,0.641397,0.639886,NaN,NaN
6,0.535856,0.538633,NaN,NaN,0.642822,0.641607,NaN,NaN
7,0.534722,0.538196,NaN,NaN,0.644279,0.644494,NaN,NaN
8,0.533713,0.537837,NaN,NaN,0.645262,0.644621,NaN,NaN
9,0.532768,0.537529,NaN,NaN,0.646032,0.645355,NaN,NaN
10,0.531905,0.537344,NaN,NaN,0.646775,0.646216,NaN,NaN


***

In [35]:
bs_list = [32, 64, 128, 256]
hidden_list = [16, 32, 64, 128]
shuffle_list = [True, False]
n_layers_list = [1, 2, 3]
lr_list = [1e-4, (1e-3)*(1/3), (1e-3)*(2/3), 1e-3]

In [ ]:
best_loss = 999999999

for batch_size in bs_list:
    for shuffle in shuffle_list:
        train_batches = get_all_batches(batch_size, shuffle, dfnp, train_files, file_index_dict, cols_dict)
        valid_batches = get_all_batches(batch_size, shuffle, dfnp, valid_files, file_index_dict, cols_dict)
        
        for n_layers in n_layers_list:
            for hidden_size in hidden_list:
                for lr in lr_list:
                    model = LSTMNet(input_size, hidden_size, n_layers, dropout, out_features)
                    model = model.to(device)
                    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
                    criterion = nn.BCELoss(reduction="mean")
                    criterion = criterion.to(device)
                    
                    for epoch in range(1, n_epochs+1):
                        train_loss, train_acc, train_conf_mat = train(model, train_batches, optimizer, criterion, device)
                    valid_loss, valid_acc, valid_conf_mat = evaluate(model, valid_batches, criterion, device)
                                        
                    print(f"loss: {valid_loss:.3f}, acc: {valid_acc*100:.2f}%, batch_size:{batch_size}, shuffle:{shuffle}, n_layers:{n_layers}, hidden_size:{hidden_size}, lr:{lr}")
                    
                    if valid_loss < best_loss:
                        best_loss = valid_loss

loss: 0.546, acc: 63.59%, batch_size:32, shuffle:True, n_layers:1, hidden_size:16, lr:0.0001
loss: 0.540, acc: 64.13%, batch_size:32, shuffle:True, n_layers:1, hidden_size:16, lr:0.0003333333333333333
loss: 0.545, acc: 63.51%, batch_size:32, shuffle:True, n_layers:1, hidden_size:16, lr:0.0006666666666666666
loss: 0.548, acc: 63.63%, batch_size:32, shuffle:True, n_layers:1, hidden_size:16, lr:0.001
loss: 0.545, acc: 64.03%, batch_size:32, shuffle:True, n_layers:1, hidden_size:32, lr:0.0001
loss: 0.546, acc: 63.70%, batch_size:32, shuffle:True, n_layers:1, hidden_size:32, lr:0.0003333333333333333
loss: 0.562, acc: 62.56%, batch_size:32, shuffle:True, n_layers:1, hidden_size:32, lr:0.0006666666666666666
loss: 0.564, acc: 63.37%, batch_size:32, shuffle:True, n_layers:1, hidden_size:32, lr:0.001
loss: 0.549, acc: 63.78%, batch_size:32, shuffle:True, n_layers:1, hidden_size:64, lr:0.0001
loss: 0.563, acc: 62.95%, batch_size:32, shuffle:True, n_layers:1, hidden_size:64, lr:0.00033333333333333

loss: 0.559, acc: 62.96%, batch_size:32, shuffle:False, n_layers:3, hidden_size:16, lr:0.0006666666666666666
loss: 0.576, acc: 62.19%, batch_size:32, shuffle:False, n_layers:3, hidden_size:16, lr:0.001
loss: 0.544, acc: 63.84%, batch_size:32, shuffle:False, n_layers:3, hidden_size:32, lr:0.0001
loss: 0.552, acc: 63.13%, batch_size:32, shuffle:False, n_layers:3, hidden_size:32, lr:0.0003333333333333333
loss: 0.589, acc: 61.54%, batch_size:32, shuffle:False, n_layers:3, hidden_size:32, lr:0.0006666666666666666
loss: 0.626, acc: 61.13%, batch_size:32, shuffle:False, n_layers:3, hidden_size:32, lr:0.001
loss: 0.547, acc: 63.58%, batch_size:32, shuffle:False, n_layers:3, hidden_size:64, lr:0.0001
loss: 0.616, acc: 60.27%, batch_size:32, shuffle:False, n_layers:3, hidden_size:64, lr:0.0003333333333333333
loss: 0.669, acc: 58.79%, batch_size:32, shuffle:False, n_layers:3, hidden_size:64, lr:0.0006666666666666666
loss: 0.569, acc: 62.67%, batch_size:32, shuffle:False, n_layers:3, hidden_size:6

loss: 0.545, acc: 63.84%, batch_size:64, shuffle:False, n_layers:2, hidden_size:32, lr:0.0001
loss: 0.545, acc: 63.46%, batch_size:64, shuffle:False, n_layers:2, hidden_size:32, lr:0.0003333333333333333
loss: 0.563, acc: 62.67%, batch_size:64, shuffle:False, n_layers:2, hidden_size:32, lr:0.0006666666666666666
loss: 0.576, acc: 61.73%, batch_size:64, shuffle:False, n_layers:2, hidden_size:32, lr:0.001
loss: 0.543, acc: 63.99%, batch_size:64, shuffle:False, n_layers:2, hidden_size:64, lr:0.0001
loss: 0.571, acc: 62.16%, batch_size:64, shuffle:False, n_layers:2, hidden_size:64, lr:0.0003333333333333333
loss: 0.603, acc: 61.61%, batch_size:64, shuffle:False, n_layers:2, hidden_size:64, lr:0.0006666666666666666
loss: 0.675, acc: 58.56%, batch_size:64, shuffle:False, n_layers:2, hidden_size:64, lr:0.001
loss: 0.556, acc: 62.59%, batch_size:64, shuffle:False, n_layers:2, hidden_size:128, lr:0.0001
loss: 0.598, acc: 61.50%, batch_size:64, shuffle:False, n_layers:2, hidden_size:128, lr:0.00033

loss: 0.545, acc: 63.70%, batch_size:128, shuffle:False, n_layers:1, hidden_size:32, lr:0.0003333333333333333
loss: 0.540, acc: 64.13%, batch_size:128, shuffle:False, n_layers:1, hidden_size:32, lr:0.0006666666666666666
loss: 0.549, acc: 63.14%, batch_size:128, shuffle:False, n_layers:1, hidden_size:32, lr:0.001
loss: 0.548, acc: 63.69%, batch_size:128, shuffle:False, n_layers:1, hidden_size:64, lr:0.0001
loss: 0.546, acc: 63.83%, batch_size:128, shuffle:False, n_layers:1, hidden_size:64, lr:0.0003333333333333333
loss: 0.557, acc: 62.89%, batch_size:128, shuffle:False, n_layers:1, hidden_size:64, lr:0.0006666666666666666
loss: 0.568, acc: 62.70%, batch_size:128, shuffle:False, n_layers:1, hidden_size:64, lr:0.001
loss: 0.546, acc: 63.85%, batch_size:128, shuffle:False, n_layers:1, hidden_size:128, lr:0.0001
loss: 0.558, acc: 62.80%, batch_size:128, shuffle:False, n_layers:1, hidden_size:128, lr:0.0003333333333333333
loss: 0.567, acc: 63.08%, batch_size:128, shuffle:False, n_layers:1, h

loss: 0.540, acc: 64.01%, batch_size:256, shuffle:True, n_layers:3, hidden_size:32, lr:0.0006666666666666666
loss: 0.542, acc: 63.56%, batch_size:256, shuffle:True, n_layers:3, hidden_size:32, lr:0.001
loss: 0.549, acc: 63.55%, batch_size:256, shuffle:True, n_layers:3, hidden_size:64, lr:0.0001
loss: 0.542, acc: 63.90%, batch_size:256, shuffle:True, n_layers:3, hidden_size:64, lr:0.0003333333333333333
loss: 0.554, acc: 63.06%, batch_size:256, shuffle:True, n_layers:3, hidden_size:64, lr:0.0006666666666666666
loss: 0.569, acc: 62.65%, batch_size:256, shuffle:True, n_layers:3, hidden_size:64, lr:0.001
loss: 0.549, acc: 63.71%, batch_size:256, shuffle:True, n_layers:3, hidden_size:128, lr:0.0001
loss: 0.547, acc: 63.09%, batch_size:256, shuffle:True, n_layers:3, hidden_size:128, lr:0.0003333333333333333
loss: 0.647, acc: 61.40%, batch_size:256, shuffle:True, n_layers:3, hidden_size:128, lr:0.0006666666666666666
loss: 0.654, acc: 59.80%, batch_size:256, shuffle:True, n_layers:3, hidden_siz

loss: 0.537, acc: 64.35%, batch_size:256, shuffle:True, n_layers:2, hidden_size:16, lr:0.001

***

In [53]:
device = torch.device("cuda:0")
input_size = len(Xcols)
dropout = 0.0
out_features = 1
n_epochs = 15

In [54]:
batch_size = 256
shuffle = True
n_layers = 2
hidden_size = 16
lr = 0.001

In [55]:
train_batches = get_all_batches(batch_size, shuffle, dfnp, train_files, file_index_dict, cols_dict)
valid_batches = get_all_batches(batch_size, shuffle, dfnp, valid_files, file_index_dict, cols_dict)
test_batches = get_all_batches(batch_size, shuffle, dfnp, test_files, file_index_dict, cols_dict)

In [56]:
len(train_batches), len(valid_batches), len(train_batches+valid_batches)

(852, 196, 1048)

In [57]:
model = LSTMNet(input_size, hidden_size, n_layers, dropout, out_features)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
criterion = nn.BCELoss(reduction="mean")
criterion = criterion.to(device)

In [58]:
for epoch in range(1,n_epochs+1):
    train_loss, train_acc, train_conf_mat = train(model, train_batches+valid_batches, optimizer, criterion, device)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

    test_loss, test_acc, test_conf_mat = evaluate(model, test_batches, criterion, device)
    print(f'\tTest Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')
    print()
    
    perf_log.loc[epoch, ["Train+Valid Loss", "Train+Valid Acc"]] = [train_loss, train_acc]

    perf_log.loc[epoch, ["Test Loss", "Test Acc"]] = [test_loss, test_acc]

	Train Loss: 0.557 | Train Acc: 62.41%
	Test Loss: 0.551 |  Test Acc: 63.55%

	Train Loss: 0.547 | Train Acc: 63.77%
	Test Loss: 0.547 |  Test Acc: 63.90%

	Train Loss: 0.542 | Train Acc: 64.01%
	Test Loss: 0.540 |  Test Acc: 64.00%

	Train Loss: 0.537 | Train Acc: 64.20%
	Test Loss: 0.538 |  Test Acc: 64.02%

	Train Loss: 0.536 | Train Acc: 64.30%
	Test Loss: 0.536 |  Test Acc: 64.19%

	Train Loss: 0.534 | Train Acc: 64.41%
	Test Loss: 0.535 |  Test Acc: 64.35%

	Train Loss: 0.533 | Train Acc: 64.57%
	Test Loss: 0.534 |  Test Acc: 64.38%

	Train Loss: 0.532 | Train Acc: 64.67%
	Test Loss: 0.534 |  Test Acc: 64.51%

	Train Loss: 0.531 | Train Acc: 64.81%
	Test Loss: 0.534 |  Test Acc: 64.56%

	Train Loss: 0.530 | Train Acc: 64.84%
	Test Loss: 0.534 |  Test Acc: 64.51%

	Train Loss: 0.530 | Train Acc: 64.90%
	Test Loss: 0.535 |  Test Acc: 64.47%

	Train Loss: 0.529 | Train Acc: 64.94%
	Test Loss: 0.535 |  Test Acc: 64.40%

	Train Loss: 0.529 | Train Acc: 64.97%
	Test Loss: 0.536 |  Test

In [59]:
perf_log

,Train Loss,Valid Loss,Train+Valid Loss,Test Loss,Train Acc,Valid Acc,Train+Valid Acc,Test Acc
1,0.560175,0.553076,0.556589,0.551334,0.616096,0.634593,0.62412,0.635548
2,0.548298,0.550298,0.546754,0.546874,0.634768,0.636138,0.637697,0.638986
3,0.545292,0.545981,0.541988,0.540044,0.635576,0.636543,0.640132,0.639997
4,0.540648,0.54049,0.537452,0.537514,0.638334,0.637733,0.641973,0.640199
5,0.537382,0.53929,0.535564,0.536159,0.641397,0.639886,0.643033,0.641943
6,0.535856,0.538633,0.534133,0.53511,0.642822,0.641607,0.64407,0.64346
7,0.534722,0.538196,0.532902,0.534352,0.644279,0.644494,0.645705,0.643764
8,0.533713,0.537837,0.531837,0.533929,0.645262,0.644621,0.646707,0.645129
9,0.532768,0.537529,0.530947,0.533934,0.646032,0.645355,0.648144,0.645584
10,0.531905,0.537344,0.530207,0.534323,0.646775,0.646216,0.648432,0.645078


In [60]:
perf_log.to_csv("loss_acc_log.csv", index=True)

***

In [42]:
((test_conf_mat/test_conf_mat.sum().sum())*100).round(2)

,Actual Positive,Actual Negative
Predicted Positive,27.54,18.94
Predicted Negative,16.53,36.99


In [43]:
# negative = Terrorist
# positive = CounterTerrorist

test_conf_mat.columns = ["CT's Win Next Round", "T's Win Next Round"]
test_conf_mat.index = ["Predicted CT's Win Next", "Predicted T's Win Next"]
test_conf_mat

,CT's Win Next Round,T's Win Next Round
Predicted CT's Win Next,10893.0,7491.0
Predicted T's Win Next,6540.0,14634.0


In [44]:
test_conf_mat = ((test_conf_mat/test_conf_mat.sum().sum())*100).round(2)
test_conf_mat

,CT's Win Next Round,T's Win Next Round
Predicted CT's Win Next,27.54,18.94
Predicted T's Win Next,16.53,36.99


In [45]:
import pickle
with open("test_model_LSTM.pickle", "wb") as f:
    pickle.dump(model, f)

***

In [46]:
import pickle
with open("test_model_LSTM.pickle", "rb") as f:
    model = pickle.load(f)

In [47]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
criterion = nn.BCELoss(reduction="mean")
criterion = criterion.to(device)

In [61]:
grad_batches = []
model.train()
for batch in test_batches:
    X = batch[0]
    y = batch[1]
    
    X = X.to(device)
    X.requires_grad = True
    
    optimizer.zero_grad()
    
    predictions = model(X).squeeze()

    criterion.weight = torch.tensor([0.0 if x == 0.5 else 1.0 for x in y.flatten()], device=device)

    y = y.to(device)
    loss = criterion(predictions, y)
    loss.backward()
    
    grad_batches.append(X.grad)
    
len(grad_batches)

196

In [62]:
len(test_batches)

196

In [63]:
batch1 = grad_batches[0]
type(batch1)

torch.Tensor

In [64]:
batch1.shape

torch.Size([256, 1, 66])

In [65]:
grouped_grad_batches = []
for L in range(1,30):
    L_batches = [batch for batch in grad_batches if batch.shape[1] == L]
    grouped_grad_batches.append(torch.cat(L_batches))

grouped_grad_batches

[tensor([[[-3.0475e-06, -2.6131e-04,  1.6998e-04,  ..., -4.7292e-05,
            1.5880e-04, -3.4269e-04]],
 
         [[-4.0157e-05,  4.8798e-04, -9.4070e-04,  ...,  4.3133e-05,
           -3.5902e-04,  6.6034e-04]],
 
         [[ 9.4600e-05, -3.0453e-04,  7.5915e-04,  ..., -4.2990e-05,
            2.5020e-04, -4.4791e-04]],
 
         ...,
 
         [[ 2.1432e-04, -6.8990e-04,  1.7198e-03,  ..., -9.7394e-05,
            5.6682e-04, -1.0147e-03]],
 
         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]],
 
         [[-1.0684e-04, -5.7980e-04,  3.2947e-04,  ..., -1.4862e-04,
            3.9138e-04, -9.4486e-04]]], device='cuda:0'),
 tensor([[[ 6.6318e-05,  6.1915e-04,  3.2665e-04,  ..., -3.5495e-04,
           -7.5853e-04,  3.1483e-04],
          [ 3.8925e-05,  8.5146e-04, -7.6230e-04,  ...,  1.6559e-04,
           -7.4717e-05,  1.6181e-03]],
 
         [[ 4.8610e-05,  3.4326e-05,  2.6693e-04,  ...,  4.4094e-05,
           -1.8055e-0

In [66]:
grouped_grad_batches[0].shape

torch.Size([1905, 1, 66])

In [67]:
grouped_grad_batches[1].shape

torch.Size([1905, 2, 66])

In [68]:
# aggregate along time dimension using mean
aggregated_batches = []
for batch in grouped_grad_batches:
    agg_mean = torch.mean(batch, dim=1)
    aggregated_batches.append(agg_mean)
    
mean_grad_tensor = torch.cat(aggregated_batches)
mean_grad_tensor.shape

torch.Size([46259, 66])

In [69]:
# aggregate along sample dimension using mean
grad_feat_imp = torch.mean(mean_grad_tensor, dim=0)
grad_feat_imp.shape

torch.Size([66])

In [70]:
grad_feat_imp_df = pd.Series(grad_feat_imp.cpu().numpy(), index=Xcols)
grad_feat_imp_df

ct_wp_type_Pistol_kills   -1.912279e-07
t_wp_type_Rifle_damage    -4.962582e-06
ct_wp_type_Heavy_kills     6.043757e-06
t_wp_type_Grenade_kills   -2.448543e-06
ct_Molotov_hits            3.381215e-06
                               ...     
t_wp_type_Pistol_damage   -1.316303e-05
t_HE_hits                  1.021523e-06
map_de_nuke               -3.890468e-08
t_survived                 3.126637e-06
t_wp_type_Unkown_damage   -6.716970e-06
Length: 66, dtype: float32

In [86]:
grad_feat_imp_df.abs().sort_values(ascending=False)[0:6]

winner_side                    0.000022
t_wp_type_Pistol_damage        0.000013
t_wp_type_Pistol_kills         0.000013
length_seconds                 0.000010
ct_wp_type_Equipment_damage    0.000009
ct_Incendiary_hits             0.000009
dtype: float32

In [87]:
grad_feat_imp_top6 = grad_feat_imp_df.abs().sort_values(ascending=False)[0:6]

In [88]:
grad_feat_imp_top6.to_csv("grad_feat_imp.csv")

***

In [73]:
dfnp.shape

(375332, 69)

In [74]:
[x for x in Xcols]

['ct_wp_type_Pistol_kills',
 't_wp_type_Rifle_damage',
 'ct_wp_type_Heavy_kills',
 't_wp_type_Grenade_kills',
 'ct_Molotov_hits',
 't_wp_type_Equipment_damage',
 't_Incendiary_hits',
 'ct_wp_type_Equipment_kills',
 't_headshot_hits',
 't_wp_type_Grenade_damage',
 'ct_wp_type_Sniper_damage',
 't_Decoys_thrown',
 'ct_wp_type_SMG_damage',
 't_damage',
 't_eq_val',
 'ct_survived',
 't_wp_type_Sniper_damage',
 'map_de_cache',
 't_num_dmg_instances',
 'ct_wp_type_Sniper_kills',
 't_Flashes_thrown',
 'ct_num_dmg_instances',
 'map_de_inferno',
 'map_de_cbble',
 't_wp_type_Unkown_kills',
 'winner_side',
 't_wp_type_Rifle_kills',
 'ct_wp_type_Grenade_damage',
 'ct_wp_type_Unkown_damage',
 't_wp_type_Heavy_kills',
 't_wp_type_Heavy_damage',
 'ct_wp_type_Rifle_damage',
 't_wp_type_Pistol_kills',
 'ct_wp_type_Grenade_kills',
 'ct_Flashes_thrown',
 'ct_Decoys_thrown',
 'ct_Smokes_thrown',
 'ct_wp_type_Equipment_damage',
 'ct_eq_val',
 'map_de_overpass',
 'map_de_dust2',
 'ct_damage',
 'ct_Incendiary

In [75]:
perm_imp_loss = pd.Series(index=Xcols)
perm_imp_acc = pd.Series(index=Xcols)
# permute the feature within batches
for col_index, col_name in enumerate(Xcols):
#     print(f"Finding permutation importance for {col_name}")
    permuted_test_batches = []
    for batch in test_batches:
        X = batch[0]
        y = batch[1]
        perm = torch.randperm(X.shape[1])
        Xperm = torch.clone(X)
        Xperm[:,:,col_index] = Xperm[:,perm,col_index]      
        permuted_test_batches.append((Xperm, y))
        
#     print(f"X[0,:,[col_index-1, col_index, col_index+1]]: {X[0,:,[col_index-1, col_index, col_index+1]]}")
#     print(f"Xperm[0,:,[col_index-1, col_index, col_index+1]]: {Xperm[0,:,[col_index-1, col_index, col_index+1]]}")
    
    
    
    assert len(permuted_test_batches) == len(test_batches), "permuted wrong"
#     print(f"\tSuccessfully permuted")
    test_loss, test_acc, test_conf_mat = evaluate(model, permuted_test_batches, criterion, device)
    perm_imp_loss.loc[col_name] = test_loss
    perm_imp_acc.loc[col_name] = test_acc

In [76]:
perm_imp_loss

ct_wp_type_Pistol_kills    0.536178
t_wp_type_Rifle_damage     0.536800
ct_wp_type_Heavy_kills     0.535603
t_wp_type_Grenade_kills    0.535625
ct_Molotov_hits            0.535581
                             ...   
t_wp_type_Pistol_damage    0.536072
t_HE_hits                  0.535673
map_de_nuke                0.535576
t_survived                 0.536007
t_wp_type_Unkown_damage    0.535568
Length: 66, dtype: float64

In [77]:
test_loss, test_acc, test_conf_mat = evaluate(model, test_batches, criterion, device)
test_loss, test_acc

(0.5355762182449808, 0.6437130289701198)

In [78]:
perm_imp_loss = perm_imp_loss - test_loss
perm_imp_loss

ct_wp_type_Pistol_kills    0.000602
t_wp_type_Rifle_damage     0.001223
ct_wp_type_Heavy_kills     0.000027
t_wp_type_Grenade_kills    0.000049
ct_Molotov_hits            0.000005
                             ...   
t_wp_type_Pistol_damage    0.000496
t_HE_hits                  0.000096
map_de_nuke                0.000000
t_survived                 0.000431
t_wp_type_Unkown_damage   -0.000008
Length: 66, dtype: float64

In [85]:
perm_imp_loss.abs().sort_values(ascending=False)[0:6]

winner_side               0.049427
ct_eq_val                 0.004989
t_wp_type_Rifle_kills     0.002666
t_eq_val                  0.001459
t_wp_type_Sniper_kills    0.001263
t_wp_type_Rifle_damage    0.001223
dtype: float64

In [89]:
perm_imp_loss_top6 = perm_imp_loss.abs().sort_values(ascending=False)[0:6]

In [90]:
perm_imp_loss_top6.to_csv("perm_imp_loss.csv")